# BANCO BASE - EJERCICIO 1 

__30 de marzo del 2022__

Se genera el siguiente Notebook para dar respuesta a los siguientes puntos del Ejercicio 1:
    
> 1.1. ¿En qué horarios hay mayor afluencia y en qué estaciones? 
    
> 1.2. Usa un método de aprendizaje no supervisado para encontrar perfiles de uso de las estaciones. Lo que debes hacer es categorizar a las estaciones en diferentes grupos a partir de su comportamiento de entradas, salidas y tipo de usuarios.

> 1.3. Usa un modelo estadístico paramétrico para que, a partir de un análisis temporal, contestes lo siguiente:



### Importar archivos

Se importan tres archivos .csv de manera independiente, esto porque es necesario primero ver el contenido de cada archivo 

In [ ]:
%%time
import os

print(os.getcwd())

In [ ]:
%%time
import pandas as pd
import glob

path_in = '/ejercicios/Ejercicio1/input/' # el path donde se encuantran los archivos .cvs a importar 
path_out = '/ejercicios/Ejercicio1/output/' # el path donde se exportan archivos 
files = glob.glob(path_in + "/*.csv") # se pasan los paths de cada archivo a una lista 
dfs = []

for file in files:
    df = pd.read_csv(file)
    print(df.shape)
    dfs.append(df)

### Descripción de los datasets

Los datasets fueron tomados del sitio ecobici: https://www.ecobici.cdmx.gob.mx/es/informacion-del-servicio/open-data
    
Cada dataset contiene los mismos atributos, los cuales se describen a continuación: 
    
- __Genero_Usuario:__ el género del usuario (solo dos valores para masculino y femenino) 	
- __Edad_Usuario:__ la edad del usuario
- __Bici:__ el No de bicicleta 
- __Ciclo_Estacion_Retiro:__ el número de la estación de la cual se toma la bicicleta 	
- __Fecha_Retiro_:__ la fecha en que se retira la bicicleta
- __Hora_Retiro:__ la hora en que se retira la bicicleta
- __Ciclo_EstacionArribo:__ el número de estación en que se regresa la bicicleta
- __Fecha Arribo:__ la fecha en que se regresa la bicicleta
- __Hora_Arribo:__ la hora en que se regresa la bicicleta

### Información General de DFs

Se explora el contenido de los archivos para ver los atributos, el tipo de datos y los valores null en cada uno de los atributos

In [ ]:
%%time

df.head()

In [ ]:
%%time

dfs[0].info()

In [ ]:
%%time

dfs[1].info()

In [ ]:
%%time

dfs[2].info()

### Concatenar archivos

Una vez que se han validado los atributos de los tres archvios, se concatena la lista de DataFrames a un solo DataFrame para facilitar el manejo de los datos

In [ ]:
%%time

df = pd.concat(dfs, axis=0, ignore_index=True)

# para validar el tamaño del archivo concatenado contra la suma de los tres archivos importados
print('Suma de archivos separados: ', dfs[0].shape[0] + dfs[1].shape[0] + dfs[2].shape[0])
print('Archivo concatenado', df.shape)

In [ ]:
%%time

df.info()

### Llenar valores nan

Se decide reemplazar valores null en el atributo Genero Usuario, lo anterio debido a que es el atributo con el mayor número de valore null, y al quitar estos valores, se pierde información que se considera siginificativa. El valor se reemplaza por una "N" que representa neutro. 

In [ ]:
%%time

# valores null en atributo "Genero Usuario"
df.info()

### Genero Usuario sin valores Null

In [ ]:
%%time

df['Genero_Usuario'].fillna('N', inplace = True)
df.info()

### Reemplazar los demás valores null 

A partir de las distribuciones mostradas en las gráficas siguientes, se decide reemplazar los valores de los atributos: Edad_Usuario, Bici y Ciclo_EstacionArribo

In [ ]:
%%time
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize= (8, 4))
sns.distplot(df['Edad_Usuario'])

In [ ]:
%%time
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize= (8, 4))
sns.distplot(df['Bici'])

In [ ]:
%%time
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize= (8, 4))
sns.distplot(df['Ciclo_EstacionArribo'])

### Reemplazar valores null

Se decide reemplazar los valores antes mencionados a partir de la media, ya que se considera que existen sesgos en la distribución 

In [ ]:
%%time

df['Edad_Usuario'].fillna(df['Edad_Usuario'].median(), inplace = True)
df['Bici'].fillna(df['Edad_Usuario'].median(), inplace = True)
df['Ciclo_EstacionArribo'].fillna(df['Edad_Usuario'].median(), inplace = True)


print('Edad media: ', df['Edad_Usuario'].median())
print('Bici media: ', df['Bici'].median())
print('Estación media: ', df['Ciclo_EstacionArribo'].median())

In [ ]:
df.info()

# Pregunta 1.1

¿En qué horarios hay mayor afluencia y en qué estaciones?

Para responder a la pregunta 1.1, es necesario primero transformar el atributo de horarios a un formato de horario determinado, en este caso se elige un formato de hora, es decir, de 0-24. La razón de esto es que se considera que en lapsos de una hora sería posible establecer mejor la afluencia. 

__Nota:__ La pregunta 1.1 menciona la mayor afluencia en determinado horario, sin embargo; por afluencia se entiende la mayor cantidad de personas a una determinada hora, pero esto puede ser al momento de ir por la bicicleta o al momento de regresarla, por lo cual se toman los dos valores para dar formato al atributo del horario. Así mismo, la pregunta no especifica si se refiere a la mayor afluencia en estaciones de forma general, o a la mayor afleuncia en estaciones a partir de las horas de mayor afluencia.  

In [ ]:
%%time

# se genera dos nuevas columnas en el DF con la hora de arribo y retiro 
df['Hora_Arribo_formato'] = pd.to_datetime(df['Hora_Arribo'], format='%H:%M:%S').dt.hour
df['Hora_Retiro_formato'] = pd.to_datetime(df['Hora_Retiro'], format='%H:%M:%S').dt.hour

df.shape

In [ ]:
%%time

df['Hora_Arribo_formato'] = df['Hora_Arribo_formato'].astype('float64')
df['Hora_Retiro_formato'] = df['Hora_Retiro_formato'].astype('float64')

### Distribución de la Hoar de Arribo

Esto se realiza debido a que mas adelante se utlizará este dato, y es necesario reemplezar los valores null en este atributo 

In [ ]:
%%time
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize= (8, 4))
sns.distplot(df['Hora_Arribo_formato'])

### Reemplazar valor null 

Se reemplazan valores null en la Hora_Arribo_formato a partir de la media 

In [ ]:
%%time

df['Hora_Arribo_formato'].fillna(df['Hora_Arribo_formato'].median(), inplace = True)

### Agrupar y contar 

Se realiza un agrupamiento y conteo a partir del horario de arribo y retiro y el atributo "Ciclo_EstacionArribo", el cual contienen la estación, con esto se puede saber el horario con la mayor afluencia, se sacan tambien porcentajes

In [ ]:
%%time

# primer agrupamiento por la hora de arribo
df_horaArribo = df.groupby(['Hora_Arribo_formato'])['Ciclo_EstacionArribo'].count().reset_index().copy()

# se genera una columna con los porcentajes 
df_horaArribo['pct'] = [(i * 1.0) / df_horaArribo['Ciclo_EstacionArribo'].sum() for i in df_horaArribo['Ciclo_EstacionArribo']]

# se muestran los valores de mayor a menor 
df_horaArribo.sort_values(by='pct', ascending = False)

In [ ]:
%%time

# segundo agrupamiento por la hora de retiro
df_horaRetiro = df.groupby(['Hora_Retiro_formato'])['Ciclo_Estacion_Retiro'].count().reset_index().copy()

# se genera una columna con los porcentajes 
df_horaRetiro['pct'] = [(i * 1.0) / df_horaRetiro['Ciclo_Estacion_Retiro'].sum() for i in df_horaRetiro['Ciclo_Estacion_Retiro']]

# se muestran los valores de mayor a menor 
df_horaRetiro.sort_values(by='pct', ascending = False)

### Respuesta 1.1

Las dos agrupaciones anteriores responden la primer parte de la pregunta, que es ¿en que horario hay mayor afluencia?

- __horario mayor afluencia:__ es de las 13:00  hasta las 20:00 horas, es decir que los horarios con mayor afluencia son 13:00, 14:00, 15:00, 16:00, 17:00, 18:00 y 19:00 horas, se encuentran dos __outliers__ tanto para los horarios de entrada como de salida con mas afluencia que son las 9:00 y 8:00 horas. 

La razón por la cual se consideran outliers estos dos horarios (8 y 9 horas) es debido a que ambos horarios tienen una afluencia similar o incluso mayor a los encontrados en el horario (13-20 horas), además, estos son horarios de la mañana (8:00 a 10:00 horas). Esto permite inferir que existe una afleuncia importante de usuarios en estas dos horas de la mañana, además de la afluencia encontrada en el horario de 13-20 horas. 


### Parte dos Respuesta 1.1

Para responder a la segunda parte de la pregunta 1.1 ¿En qué estaciones?, se realiza una nueva agrupación tomando como input el dato del horario de mayor afluencia, que en este caso es de las 13:00 y menor a las 20:00 horas

In [ ]:
%%time

# se deja solo los horarios de mayor afluencia 
df_mAfluenciaA = df[df['Hora_Arribo_formato'] >= 13.0].copy()
df_mAfluenciaA = df_mAfluenciaA[df_mAfluenciaA['Hora_Arribo_formato'] < 20.0].copy()

print(df_mAfluenciaA.shape)
print(df_mAfluenciaA['Hora_Arribo_formato'].unique())

In [ ]:
%%time

# tercer agrupamiento por la estación y por arribo
df_mAfEstacionA = df_mAfluenciaA.groupby(['Ciclo_EstacionArribo'])['Hora_Arribo_formato'].count().reset_index().copy()
df_mAfEstacionA.sort_values(by='Hora_Arribo_formato', ascending = False)

In [ ]:
%%time

# se deja solo los horarios de mayor afluencia 
df_mAfluenciaR = df[df['Hora_Retiro_formato'] >= 13.0].copy()
df_mAfluenciaR = df_mAfluenciaR[df_mAfluenciaR['Hora_Retiro_formato'] < 20.0].copy()

print(df_mAfluenciaR.shape)
print(df_mAfluenciaR['Hora_Retiro_formato'].unique())

In [ ]:
%%time

# cuarto agrupamiento por la estación y el retiro 
df_mAfEstacionR = df_mAfluenciaR.groupby(['Ciclo_Estacion_Retiro'])['Hora_Retiro_formato'].count().reset_index().copy()
df_mAfEstacionR.sort_values(by='Hora_Retiro_formato', ascending = False)

# Resumen Respuesta 1.1

La mayor afluencia se da en los horarios de las 13:00 hasta antes de las 20:00 horas, es decir que los horarios con mayor afluencia tanto para el arribo como para el retiro son: 

- 13:00, 14:00, 15:00, 16:00, 17:00, 18:00 y 19:00 horas, 
    
Se decide tomar este rango de horarios porque en estos horarios se concentra aproximadamente cerca del 42% de la afluencia total. 

Así mismo, se encuentran dos outliers tanto para los horarios de arribo como de retiro con mas afluencia que son las 9:00 horas para los arribos y las 8:00 horas para los retiros, se decide no considerarlos ya que representan aproximadamente el %6 del total respectivamente. 

Dentro del rango de horario de mayor afluencia mencionado anteriormente, las estaciones con mayor afluencia son: 

- 271, 266, 1, 64 y 27 en el caso de los arribos
- 1, 27, 64, 41 y 182 en el caso de los retiros 



# Pregunta 1.2.


Método de aprendizaje no supervisado para encontrar perfiles de uso de las estaciones

Para resolver esta pregunta se siguen los siguientes pasos. 

>__1. Elegir modelo:__ se elige un modelo de agrupamiento (clustering) para representar los datos, ya que se considera que un agrupamiento es la mejor opción para generar perfiles de usuarios, el algoritmo se espcifica más adelante

>__2. Datos categóricos:__ convertir los datos categoricos a numéricos, esto porque para alimentar un modelo de agrupamiento, es necesairo que los datos esten en un fromato numérico 

>__3. Normalizar el tipo de dato:__ de origen, existen datos datetime, str, float64 e incluso int, el cual se genero al numericalizar el sexo de los usuarios, por lo cual, se elige un formato de numero para todos los datos (float64), ya que otros datos de origen tienen este tipo de dato es un formato con el cual se puede alimentar el modelo de agrupamiento 

>__4. Elegir el algoritmo:__ se elige KMeans por ser un algoritmo simple, que no utliza muchos recursos, y que permite de manera general resolver la pregunta 

> __5. Determinar número de grupos:__ ya que se usa KMeans, es necesario definir el número "k", esto se hace a aprtir del método Elbow

### Convertir datos categóricos

Se pasa el atributo "Genero_Usuario" a número para poder trabajar con el dataset 

In [ ]:
%%time
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() # instancia del label encoder 
df['Genero_Usuario_num'] = le.fit_transform(df['Genero_Usuario']) # pasar a número los datos categóricos 
df['Genero_Usuario_num'] = df['Genero_Usuario_num'].astype('float64') # pasar el tipo de dato a float64

df.shape

### Pasar a nuevo DF 

Se generan dos dataframes nuevos separando los datos de arribo y retiro 

In [ ]:
%%time

dfArribo = df[['Genero_Usuario_num', 'Edad_Usuario', 'Bici', 'Ciclo_EstacionArribo', 'Hora_Arribo_formato']].copy()
dfRetiro = df[['Genero_Usuario_num', 'Edad_Usuario', 'Bici', 'Ciclo_Estacion_Retiro', 'Hora_Retiro_formato']].copy()

print(dfArribo.shape)
print(dfRetiro.shape)

### Método Elbow

Se utiliza este método para elegir el número optimo de clústers. La parte de la gráfica donde la línea es menos suave o cambia abruptamente es lo que forma un “codo”, ese número de clúster ayudará al momento de clasificar los datos.

In [ ]:
%%time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

k1 = []
inertia_s1 = []

for i in tqdm(range(2,26)):
    k1.append(i)
    kmeans1 = KMeans(n_clusters=i, random_state=125, max_iter=100).fit(dfArribo)
    inertia_s1.append(kmeans1.inertia_)

# plot
plt.figure(figsize=(15,5))
plt.plot(k1, inertia_s1,color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize=10)
plt.title('Inertia (SSE) vs. K Value')
plt.xlabel('K')
plt.ylabel('Inertia score (SSE)')

### Generar cluster KMeans

Se genera un cluster con k=5, esto porque se considera que la gráfica anterior cambia su curso de forma más abrupta entre el 4 y 5 del eje x

In [ ]:
%%time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=5)
dfArribo['cluster'] = cluster.fit_predict(dfArribo)

In [ ]:
%%time
dfArribo.head()

### Plots

Se generan plots para ver la distribución de los datos, y proceder así a contestar la pregunta 1.2

__Nota:__ Al correr la celda de abajo se puede ralentizar un poco el notebook, esto debido a que es un plot interactivo 

In [ ]:
%%time
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# creating subplot for the four attributes
fig = make_subplots(rows=3, cols=2, subplot_titles=("Genero_Usuario", "Edad_Usuario", 
                                                    "Estacion Arribo", "Hora Arribo", "Clusters"))

# volin plot para el género
fig.append_trace(go.Violin(x=dfArribo['Genero_Usuario_num'], name='Genero'), row=1, col=1)

# violin plot para la edad
fig.append_trace(go.Violin(x=dfArribo['Edad_Usuario'], name='Edad'), row=1, col=2)

# violin plot para estación arribo
fig.append_trace(go.Violin(x=dfArribo['Ciclo_EstacionArribo'], name='Estacion Arribo'), row=2, col=1)

# violin plot para hora arribo
fig.append_trace(go.Violin(x=dfArribo['Hora_Arribo_formato'], name='Hora Arribo'), row=2, col=2)

# violin plot para cluster
fig.append_trace(go.Violin(x=dfArribo['cluster'], name='Clusters'), row=3, col=1)

# mostrar el plot
fig.update_layout(height=600, width=900, title_text="Violin Subplots", template="simple_white")
fig.show()

### Agrupar 

Se realizna un par de agrupaciones para generar inferencias sobre los clusters

In [ ]:
%%time

df_1 = dfArribo.groupby(['cluster', 'Genero_Usuario_num'])['Ciclo_EstacionArribo'].size().reset_index()
df_2 = dfArribo.groupby(['Ciclo_EstacionArribo', 'cluster', 'Edad_Usuario'])['Genero_Usuario_num'].count().reset_index()

In [ ]:
%%time
import plotly.express as px

fig = px.bar(df_1, x='cluster', y='Ciclo_EstacionArribo', color = 'Genero_Usuario_num', width=500, height=300)
fig.show()

In [ ]:
%%time
import plotly.express as px


fig = px.scatter(dfArribo, x='Ciclo_EstacionArribo', y='Edad_Usuario', color = 'Genero_Usuario_num', 
                 facet_col="cluster",  width=900, height=600)
fig.show()

# Resumen Respuesta 1.2

1.2 Método de aprendizaje no supervizado para encontrar perfiles de eusuarios

>a. Explica qué método utilizaste y porqué: 
>>el método utilizado es KMeans, la razón es que este es un
algoritmo para generar clusters a partir de los datos, así mismo es un modelo sencillo de implementar y 
bajo en costo computacional.
	
>b. Describe detalladamente como decidiste el número de grupos. ¿Que criterio elegiste?, 
¿Porque? y de ¿que tipo?
>>El número de grupos se eleigio a partir del uso del método Elbow, siendo esto un método cuantitativo, 
la razón de elegirlo, es que es un método estandarizado y probado que permite conocer el número 
óptimo de clusters, la forma de hacerlo es graficando y visualizando una línea que cambia abruptamente 
formando aís un “codo”, ese cambio abrupto, es el número de clústers

>c. De los grupos encontrados, describe las características que se pueden inferir de ellos
>>De manera general se puede inferir que a partir de los clusters los hombres son los que utilizan 
	más el servicio a lo largo de las estaciones, casi 3 veces más que las mujeres, y que la distribución por 
	edad y estación, son también los hombres quienes hacen más uso del servicio. 
    Por último, en la distribución total, hay un mayor uso en el rango de edades 20-40 años, una mayor 
	concentración de usuarios en las estaciones 1-200, y una disminución significativa en el uso del 
	servicio en los horarios 00:00-07:00 horas

# 1.3 Modelo estadístico paramétrico

### Importar DF

Se importa un DF exportado en la pregunta 1.1 y 1.2, esto para no tener que correr el notebook desde el inicio y poder trabajar más rápido en esta sección, la exportación del archivo no se coloca en el notebook, pero el dataset si esta disponible 

In [ ]:
%%time
import pandas as pd

path_in = '/ejercicios/Ejercicio1/input/' # el path donde se encuantran los archivos .cvs a importar 
path_out = '/ejercicios/Ejercicio1/output/' # el path donde se exportan archivos 

df = pd.read_csv(path_out + 'dfConcat.csv')
df.shape

### Tipo de datos

Se unifican los tipos de datos, ya que las fechas y strings aparecen como objeto 

In [ ]:
%%time
import datetime

##funcion para convertir los objetas en tipo de dato datetime
df['Genero_Usuario'] = df['Genero_Usuario'].astype(str)
df['Fecha_Retiro'] = pd.to_datetime(df['Fecha_Retiro'], format="%d/%m/%Y")
df['Fecha Arribo'] = pd.to_datetime(df['Fecha Arribo'], format="%d/%m/%Y")

### LLenar valores null 

Se llenan los valores null con la media para los atributos a utilizar

In [ ]:
%%time
import numpy as np

df['Fecha_Retiro'].fillna(df['Fecha_Retiro'].median(), inplace = True)
df['Ciclo_Estacion_Retiro'].fillna(df['Ciclo_Estacion_Retiro'].median(), inplace = True)

### Sort

Se acomoda el DF por fecha

In [ ]:
%%time

df.sort_values(by = 'Fecha_Retiro', inplace = True)

### Pasar datos a utilizar 

In [ ]:
%%time

dfp = df[['Fecha_Retiro', 'Ciclo_Estacion_Retiro']].copy()

### Agrupar 

In [ ]:
%%time

ctdf = (dfp.reset_index()
          .groupby(['Fecha_Retiro','Ciclo_Estacion_Retiro'], as_index=False).count().rename(columns={'index':'ct'}))

# Resúmen respuesta 1.3


Se generan plots para visualizar cada dos estaciones la tendencia los cuales se muestran a continuación, se colocan aquí las respuestas a cada una de las preguntas: 

- ¿En qué estaciones puedes observar una tendencia de uso a la alta?
    - Estaciones 3, 5, 7, 8, 9, 12, 13, 15, 20, 24
    
- ¿En qué estaciones puedes observar una tendencia de uso a la baja?
    - Estaciones 1, 2, 4, 6, 10, 11, 14, 16, 17, 18
    
- Demuestra tus conclusiones visualmente, e.g. puedes graficar las diez estaciones con mayor tendencia de uso a la alta y las diez estaciones con menortendencia de uso a la baja.
    - Se muestran en los siguientes gráficos

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn = ctdf[ctdf['Ciclo_Estacion_Retiro'] < 3].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 3].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 5].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 5].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 7].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 7].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 9].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 9].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 11].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 11].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 13].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 13].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 15].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 15].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 17].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 17].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 19].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 19].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 21].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)

In [ ]:
%%time
import seaborn as sns
from matplotlib import pyplot as plt

nn1 = ctdf[ctdf['Ciclo_Estacion_Retiro'] >= 23].copy()
nn1 = nn1[nn1['Ciclo_Estacion_Retiro'] < 25].copy()

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize =(10, 5))

# key gives the group name (i.e. category), data gives the actual values
for key, data in nn1.groupby('Ciclo_Estacion_Retiro'):

    data.plot(x='Fecha_Retiro', y='ct', ax=ax, label=key)